This notebook is highly inspired from 
- [LSTM Text Generation](https://github.com/Lasagne/Recipes/blob/master/examples/lstm_text_generation.py)
- [Lasagne doc about recurrent](http://lasagne.readthedocs.io/en/latest/modules/layers/recurrent.html)

In [1]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
seed = 1
lasagne.random.set_rng(np.random.RandomState(seed))

Using cuDNN version 6021 on context None
Mapped name None to device cuda0: GeForce GTX TITAN Black (0000:02:00.0)


# Hyperparameters

The following are hyperparameters, except for `PRINT_FREQ`, that will have an impact on the learning algorithm.

In [33]:
# Sequence Length
SEQ_LENGTH = 20

# Number of units in the hidden (LSTM) layers
DEPTH = 2
N_HIDDEN = 512
NON_LINEARITY = lasagne.nonlinearities.rectify

# All gradients above this will be clipped
GRAD_CLIP = 100

# Number of epochs to train the net
NUM_EPOCHS = 50

# Optimization learning rate
LEARNING_RATE = 0.01

# Batch Size
BATCH_SIZE = 128

# How often should we check the output?
PRINT_FREQ = 500

An optimizer can be seen as a function that takes a gradient, obtained by backpropagation, and returns an update to be applied to the current parameters. Other optimizers can be found in: [optimizer reference](http://lasagne.readthedocs.io/en/latest/modules/updates.html?highlight=update).

In [34]:
my_optimizer = lambda loss, params: lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.01, momentum=0.9)

# Dataset

The following loads the dataset (the full text *Beyond Good and Evil* by Friedrich Nietzsche) directly from Internet.
You can also replace the dataset by your own by adapting the commented line.

In [35]:
import urllib.request #For downloading the sample text file. You won't need this if you are providing your own file.
try:
    in_text = urllib.request.urlopen('https://s3.amazonaws.com/text-datasets/nietzsche.txt').read()
    #in_text = open('your_file.txt', 'r').read()
    in_text = in_text.decode("utf-8")
    print(in_text[:250])
except Exception as e:
    print("Please verify the location of the input file/URL.")
    print("A sample txt file can be downloaded from https://s3.amazonaws.com/text-datasets/nietzsche.txt")
    raise IOError('Unable to Read Text')

PREFACE


SUPPOSING that Truth is a woman--what then? Is there not ground
for suspecting that all philosophers, in so far as they have been
dogmatists, have failed to understand women--that the terrible
seriousness and clumsy importunity with which t


The pre-processing consists in retrieving the list of symbols occuring in the text and to convert each of them into an unique index. This index will be used to create an one-hot representation of the symbol that will be the input of the model.

In [36]:
chars = list(set(in_text))
data_size, vocab_size = len(in_text), len(chars)
char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}
print('Number of unique symbols: {}'.format(vocab_size))
print('Number of symbols in the dataset: {}'.format(data_size))

Number of unique symbols: 84
Number of symbols in the dataset: 600893


The following auxiliary function creates a minibatch in a 3D tensor (batch_size, SEQ_LENGTH, vocab_size).
For each datapoint (fixed first coordinate of the 3D matrix), there is a matrix of dimension (SEQ_LENGTH, vocab_size)
where each line contains the one-hot vector representing the character at the associated position. Notice that the sequences have all the same length (SEQ_LENGTH), which can covers many sentences.

In [37]:
input_shape = (None, None, vocab_size)

def iterate_minibatch(
    p, 
    batch_size = BATCH_SIZE, 
    data=in_text, 
    return_target=True):
    """
    Return a minibatch compatible with the input of the model and the associated targets
    
    :type p: int
    :param The index of the character to begin to read
    :type batch_size: int
    :param The number of datapoints in the current batch
    :type data: str
    :param The whole text
    :type return_target: bool
    :param Create the targets (next character) associated to the sequences
    """
    x = np.zeros((batch_size,SEQ_LENGTH,vocab_size))
    y = np.zeros(batch_size)
    
    for n in range(batch_size):
        ptr = n
        for i in range(SEQ_LENGTH):
            x[n,i,char_to_ix[data[p+ptr+i]]] = 1.
        if(return_target):
            y[n] = char_to_ix[data[p+ptr+SEQ_LENGTH]]
    return x, np.array(y,dtype='int32')

# Model definition

Recurrent layers can be used similarly to feed-forward layers except that the input shape is expected to be (batch_size, sequence_length, num_inputs). By setting the first two dimensions as None, we are allowing them to vary. They correspond to batch size and sequence length, so we will be able to feed in batches of varying size with sequences of varying length. If `only_return_final` is set, it only returns the final sequential output (e.g. for tasks where a single target value for the entire sequence is desired). In this case, Theano makes an optimization, which saves memory. If you are working with variable size sequences, an additional parameters `masks` of size (batch_size, sequence_length) is given as a boolean mask where its entries are fixed to 0 after the end of the sequence.

In [38]:
def create_lstm(
    vocab_size,
    input_shape,
    input_var = None,
    nonlinearity = lasagne.nonlinearities.tanh,
    depth=2, 
    n_hidden=800,
    grad_clip=100):
    """
    A generic function for creating a LSTM neural network.

    :type vocab_size: int
    :param vocab_size: number of elements in the dictionary
    :type input_shape: tuple
    :param input_shape: a tuple containing the shape of the input
    :type input_var: theano.tensor.var.TensorVariable
    :param input_var: a theano symbolic variable, created automatically if None
    :type nonlinearity: lasagne.nonlinearities
    :param nonlinearity: a nonlinearity function that follows all dense layers
    :type depth: int
    :param depth: the depth of the LSTM
    :type n_hidden: int or list
    :param n_hidden: number of hidden units per LSTM cells (if int, the same for all layers)
    :type grad_clip: float
    :param grad_clip: threshold for the gradient in the LSTM
   """

    # First, we build the network, starting with an input layer
    # Recurrent layers expect input of shape
    # (batch size, SEQ_LENGTH, num_features)
    network = lasagne.layers.InputLayer(
        shape=input_shape,
        input_var=input_var
    )

    # We now build the LSTM layer
    # We clip the gradients at GRAD_CLIP to prevent the problem of exploding gradients. 
    for _ in range(depth-1):
    
        network = lasagne.layers.LSTMLayer(
            network, 
            num_units=n_hidden, 
            grad_clipping=grad_clip,
            nonlinearity=nonlinearity)

    network = lasagne.layers.LSTMLayer(
        network, 
        num_units=n_hidden, 
        grad_clipping=grad_clip,
        nonlinearity=nonlinearity,
        only_return_final=True)

    # The output the previous module, with shape (batch_size, N_HIDDEN),
    # is then passed through the softmax nonlinearity to 
    # create a probability distribution over the dictionary.
    # The output of this stage is (batch_size, vocab_size).
    network = lasagne.layers.DenseLayer(network, num_units=vocab_size, W = lasagne.init.Normal(), nonlinearity=lasagne.nonlinearities.softmax)
    return network

# Optimization

In the following, we want to maximize the probability to output the right character given the SEQ_LENGTH previous ones. To do this, we retrieve the output of our model, which is a softmax over the characters, and we compare it to the actual character of the sequence. Finally, since we are using minibatches of size `BATCH_SIZE`, we compute the mean over the examples of the minibatch.

In [39]:
# Theano tensor for the targets
input_var = T.tensor3('inputs')
target_values = T.ivector('target_output')

network = create_lstm(
    vocab_size,
    input_shape,
    input_var,
    nonlinearity=NON_LINEARITY,
    depth=DEPTH, 
    n_hidden=N_HIDDEN)

# lasagne.layers.get_output produces a variable for the output of the net
network_output = lasagne.layers.get_output(network)

# The loss function is calculated as the mean of the (categorical) cross-entropy between the prediction and target.
loss = lasagne.objectives.categorical_crossentropy(network_output, target_values).mean()

# Retrieve all the parameters of the models and create the optimizer
params = lasagne.layers.get_all_params(network,trainable=True)
updates = my_optimizer(loss, params)

In [40]:
# Theano functions for training and computing cost
print("Compiling functions ...")
train = theano.function([input_var, target_values], loss, updates=updates, allow_input_downcast=True)
compute_loss = theano.function([input_var, target_values], loss, allow_input_downcast=True)

# In order to generate text from the network, we need the probability distribution of the next character given
# the state of the network and the input (a seed).
# In order to produce the probability distribution of the prediction, we compile a function called probs.
probs = theano.function([input_var],network_output,allow_input_downcast=True)

Compiling functions ...


The next function generates text given a phrase of length at least `SEQ_LENGTH`.
The phrase is set using the variable generation_phrase.
The optional input `N` is used to set the number of characters of text to predict. 

In [ ]:
generation_phrase = "The meaning of life is " #This phrase will be used as seed to generate text.

def predict(N=200):
    """
    Output a sequence of characters of lenght N according to the current model
    
    :type N: int
    :param N: the number of characters to generate
    """
    assert(len(generation_phrase)>=SEQ_LENGTH)
    sample_ix = []
    x,_ = iterate_minibatch(len(generation_phrase)-SEQ_LENGTH, 1, generation_phrase,0)

    for i in range(N):
        # Pick the character that got assigned the highest probability
        # ix = np.argmax(probs(x).ravel())
        
        # Alternatively, to sample from the distribution instead:
        ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:SEQ_LENGTH-1,:] = x[:,1:,:]
        x[:,SEQ_LENGTH-1,:] = 0
        x[0,SEQ_LENGTH-1,sample_ix[-1]] = 1. 

    random_snippet = generation_phrase + ''.join(ix_to_char[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)

In [ ]:
print("Training ...")
print("Seed used for text generation is: " + generation_phrase)
p = 0
for it in range(int(data_size * NUM_EPOCHS / BATCH_SIZE)):
    predict() # Generate text using the p^th character as the start. 
            
    avg_cost = 0;
    for i in range(PRINT_FREQ):
        x,y = iterate_minibatch(p)

        p += SEQ_LENGTH + BATCH_SIZE - 1 
        if(p+BATCH_SIZE+SEQ_LENGTH >= data_size):
            print('Carriage Return')
            p = 0;

        avg_cost += train(x, y)
    print("Epoch {} average loss = {}".format(it*1.0*PRINT_FREQ/data_size*BATCH_SIZE, avg_cost / PRINT_FREQ))


Training ...
Seed used for text generation is: The meaning of life is 
----
 The meaning of life is .CXRQL
nVMbé]BRh;C.XDVqOæPj;hXsAKhZ=t 2Be=L6ënr)kë[;J)1cb=8yC"hÆMkcz[MHäæAWXEw5d;ææYäB(T HK1A(6Kz)8éNA 1m"ah04SZ'DQIkrOm!I]atJ9æIéR6éRI1ps!Kgy]!J[FVUM28]j)zq:tDAa9S4_NgvSëvDz(y8j=Pjé'_u1gIchOCh!=Ua6Km 
----
Epoch 0.0 average loss = 3.2360804438591004
----
 The meaning of life is  
rde  dsoaoeuteo emee rl neeist"
weraoihctwndiw uatdlG fi stecrbtm e hilfoertTat hie a oa bsnmlvlees -n  ttrr ooet igais 
hl whdpmoitett tEodcei - oal aenstR-werraNid ra dldi vasgl!oeo sth ezsdn ;id  
----
Epoch 0.10650814704115374 average loss = 3.072127217769623
----
 The meaning of life is khytutghd hoosytauob ilycs,ijiIctkenclCjeelte duthveonb,secagsannerhyd hrfgis huatamh9ease ohftrtoas chdd;oor, sostbie iom rnfmcsiraifrahessibcoddisadddnisipiad aicu-eosetp,ltmcde aamseosuRh
rireroehm 
----
Epoch 0.2130162940823075 average loss = 2.82948108959198
----
 The meaning of life is on ano ane ppureis tgisd bhere K

Epoch 2.875719970111151 average loss = 1.93361616396904
----
 The meaning of life is been cadess), of mosthers--"pecike
not bucres and entaobdep, incercion the femestimed, and ascempal and mofwer coull intonefted these-obly Pof, thouelt unsterfied. The sial
tanlem, we tex begorce ceet 
----
Epoch 2.982228117152305 average loss = 1.9495100967884065
----
 The meaning of life is themselver; we as froggensed,. "! Pemation of maro
coulself, seaf
istul"-moner, anteretion of gan usrelsy at sull acteabre
censuroo-
is ging to
the grestan
ware and ishirgreatd firdle, who fealus for  
----
Epoch 3.0887362641934586 average loss = 1.9272194876670838
----
 The meaning of life is te thorogand,
philosopher throngimes and sasplo id the grobtropst faromeding light and corn of a ghhies leady dunaked thatk apmithis on yis is plotion prantentaytry hamquth wish couth
and the ghange o 
----
Epoch 3.1952444112346123 average loss = 1.831642921924591
----
 The meaning of life is more; Une Wheped
the phoies misu

Epoch 5.857948087263456 average loss = 1.6123631613254548
----
 The meaning of life is and beciunt will repiness of that the puraes themselves exesenting, live much alliobed gijcoms of cife. It hatk the standing of a period petliest lutt, see
dintery of yeacht--it a viduue itsult of loa 
----
Epoch 5.96445623430461 average loss = 1.5841003918647767
----
 The meaning of life is dusing, theresos. Show
heredyes from shakely, there hidselving arcondisced ones) nowico of mins
higher geachous arwsy age ame oraging limins gener farnt distally, to them
nevery modicts hemprethid, sa 
----
Carriage Return
Epoch 6.070964381345764 average loss = 1.6416246755123138
----
 The meaning of life is pakts--as
all also inwellest pmodism--their sunder; canagor's and
betarely in such ane recomententure. That we than Ken-3ochalse as a persolary command cufikses of mainhel could--have, orey, and then  
----
Epoch 6.177472528386917 average loss = 1.6576904821395875
----
 The meaning of life is of flees ime see

Epoch 8.84017620441576 average loss = 1.5333437287807465
----
 The meaning of life is and inccidition,
maribal and was derity! we must Rest he coocetion of plactication have superd at a know eims--to assirably nowive, for the eist" the retiving oucking the crass with Hupod its a ; him  
----
Epoch 8.946684351456915 average loss = 1.5011750435829163
----
 The meaning of life is not and case pos of which
all lorstanding,
yow."
"Cermanyers) the spirituality and when in fher your statiffer, here is result to deperating, nor trith--THE
NODATESENONHE, which must not hang! Pad and 
----
Epoch 9.053192498498069 average loss = 1.5715941548347474
----
 The meaning of life is might, and moraligiving, moralizof. Thinkery musicarmy and ustrrismmuran,.
And goven to higher how clad,
sollific thunginggrery instinct agains with reysoghies: is those things to guoding of zoverther 
----
Epoch 9.159700645539223 average loss = 1.5356096608638763
----
 The meaning of life is tome! The
such far and the free 

Epoch 11.822404321568067 average loss = 1.4649428277015686
----
 The meaning of life is the mrabt, even in any'liutly
and welt the coure as the appearanttanded? eternals courage. There is a greing extents.
Letrusive of dieac, strongly. It is the "individually
goved unreason metaphysic, t 
----
Epoch 11.92891246860922 average loss = 1.3627339099645615
----
 The meaning of life is but fout all the Start; in die bacture. To do
momeching of inventions of which he life in the esty a ready and renutes, for hy ow to be manife. It deems a during indus simply a muty.
The exist, boo
mu 
----
Epoch 12.035420615650374 average loss = 1.3810626388788223
----
 The meaning of life is no the eir als
ruderal due that in so that the
puttly
spans,--as the least any one's
but in the world points and measities, of the connless of the being but even is magh
a wealf. The askers of his pri 
----
Carriage Return
Epoch 12.141928762691528 average loss = 1.3899180409908294
----
 The meaning of life is resulding of

...

Carriage Return
Epoch 98.3070197189849 average loss = 0.4512531133890152
----
 The meaning of life is let us be means of which would liäe. Wor, reading, that. And when think of such with it in effectives to hive by natural, be only believe, the sense of the persistently (in dividuated ale, and set the 
----
Epoch 98.41352786602606 average loss = 0.47140729683637617
----
 The meaning of life is healthy said the hew trivy hater, a princed and saying
sending
still be truth a very agess, the applops simply the Spiently itself as to phenticamy and hangs
desires and sublimest and takes, in the mi 
----
Epoch 98.52003601306721 average loss = 0.47345008319616316
----
 The meaning of life is no ediobably defence of the _morality of the appearance of Napoleon is the common
instinctive mocussibility of
Spoict, and apart from which must thin shame as living, and affair, who believe, that the 
----
Epoch 98.62654416010837 average loss = 0.4615921873152256
----
 The meaning of life is to be forevase

Epoch 101.28924783613722 average loss = 0.4547859465479851
----
 The meaning of life is ascended but have south of
such men to fremer, will to deceave on ENPOISITOMAL GULN' 1'25. Henough TrEelizing crucing soon,
spirit,
womlo
 delach-surselically salittle OUSE
to try, as the appearance
o 
----
Epoch 101.39575598317838 average loss = 0.4391180018186569
----
 The meaning of life is generally we Eugo habs; and
good and buhapety of them"-which kould many feoks, that the idious over than
the later Greak," or ever too willy and the also STACTER their laxt great mysterious, the multi 
----
Epoch 101.50226413021953 average loss = 0.44852268862724304
----
 The meaning of life is deduced soundden. It should perfain into the
world in this fact that
the world is anter_tion of experience and theologies, into the headonj--for the
saffically incled to the thing-in-itself is in the  
----
Epoch 101.60877227726068 average loss = 0.4364828981757164
----
 The meaning of life is advent a reason by rare ac

Epoch 104.27147595328952 average loss = 0.42011573833227156
----
 The meaning of life is and the
whole the beniffunates the spell of be very may not be done.


70

==Uul Vain pracqize different specied species only
have perceive for his acts
all superst to such
pleasure is the struggle be 
----
Carriage Return
Epoch 104.37798410033068 average loss = 0.4095465578734875
----
 The meaning of life is assume that "this number of Lace to, litely without this rights of their passions which
all new tirn to
benevous when it repressed froudictsy of the intellectuance of thoughts, Ithemselves
belonging o 
----
Epoch 104.48449224737183 average loss = 0.4195437232255936
----
 The meaning of life is to be spirits, even for the secret wandong of an were a history. One does--apparently he deep of it. Sature". The
mixharthous in
the philosopher? And praise, gues and insists that near,ly in the heart 
----
Epoch 104.59100039441299 average loss = 0.4344918920099735
----
 The meaning of life is you
wish l

Epoch 107.25370407044183 average loss = 0.40944551485776903
----
 The meaning of life is that which they require-thing of historical,y
in all sourariom other
habits of the scholls, would be a bewnecaive out of one's Awnord to
juslify, and perferently, he is also one real agreinate
knowled 
----
Epoch 107.36021221748298 average loss = 0.4108758084177971
----
 The meaning of life is to be "must. Winkers'ed called
"its Apart from plantic laughmer emplosed by the discupumony assumes
what does the truth? From what is moriful adjresses upon obtured is that overlooks emotionality of t 
----
Epoch 107.46672036452414 average loss = 0.41469375681877135
----
 The meaning of life is not self-Travily,
the contempt of view, wOrld patheof
manifest,
are the "bad
means: and ensy,
and crupilated in a loved!--this, have to see it higher myneplet-rare, could still determine might be
laid 
----
Epoch 107.57322851156529 average loss = 0.4141509481668472
----
 The meaning of life is neverthe explained, stuci

Epoch 110.23593218759413 average loss = 0.3946824797093868
----
 The meaning of life is ordinantly as a describab it that a manching in order to mankind and costrists ones that so much as another
part in ascultased a
ticted to a means in pows superstation, which, but reflective world-per 
----
Epoch 110.34244033463528 average loss = 0.3839768939614296
----
 The meaning of life is one, for example, be must be wholly conceded, that diliger nature
insuch a sense is expedianism of the artist, that is, the crude, ever
as disposition in man.


32

=Veriting of vain man and rational  
----
Epoch 110.44894848167644 average loss = 0.37870427921414374
----
 The meaning of life is in the idea of man that has yet been in this dangerous things that all philosophers, within themselves. It is they also in this
way ever been in distentung. It is to
bewengory with a famous.
It, and h 
----
Carriage Return
Epoch 110.5554566287176 average loss = 0.38559451350569723
----
 The meaning of life is to be hono